Import the libraries necessary to analize data

In [6]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import datetime
import random
import seaborn as sns
import plotly.express as px

from collections import defaultdict
from scipy.stats import pearsonr

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [13]:
df = pd.read_csv('df_after_dp.csv', parse_dates=['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197925 entries, 0 to 197924
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   date                            197925 non-null  datetime64[ns]
 1   state                           197925 non-null  object        
 2   city_or_county                  197925 non-null  object        
 3   latitude                        197925 non-null  float64       
 4   longitude                       197925 non-null  float64       
 5   congressional_district          197925 non-null  int64         
 6   participant_age1                135327 non-null  float64       
 7   participant_age_group1          160339 non-null  object        
 8   participant_gender1             165727 non-null  object        
 9   min_age_participants            135331 non-null  float64       
 10  avg_age_participants            135327 non-null  float64

Since we use distance metric working on numerical features we remove the categorical data. 

In [25]:
column_to_drop = ['date']
for col in df.columns:
    if df[col].dtype == 'object':
       column_to_drop.append(col)
column_to_drop

['date',
 'state',
 'city_or_county',
 'participant_age_group1',
 'participant_gender1',
 'incident_characteristics1',
 'incident_characteristics2',
 'party']

In [26]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [27]:
df_numeric = df.drop(columns=column_to_drop)
df_numeric

,latitude,longitude,congressional_district,participant_age1,min_age_participants,avg_age_participants,max_age_participants,n_participants_child,n_participants_teen,n_participants_adult,...,totalvotes,substitute_values,males_percentage,killed_percentage,injured_percentage,killed_percentage_per_incident,unharmed_percentage,year,month,day
0,39.119550,-76.708250,4,17.0,17.0,17.0,17.0,0.0,1.0,0.0,...,320650,1.0,1.000000,0.0,0.034483,0.0,0.00000,2017,2,4
1,39.122636,-76.709382,3,25.0,25.0,25.0,25.0,0.0,0.0,1.0,...,339675,1.0,0.066667,0.0,0.000000,0.0,0.00000,2016,1,14
2,36.145695,-95.934236,1,28.0,28.0,28.0,28.0,0.0,0.0,2.0,...,1,2.0,0.002506,0.0,0.000000,0.0,0.00000,2017,12,22
3,39.119550,-76.708250,3,35.0,35.0,35.0,35.0,0.0,0.0,1.0,...,215946,1.0,1.000000,0.0,0.000000,0.0,0.00000,2015,3,3
4,38.970380,-76.502863,4,50.0,50.0,50.0,50.0,0.0,0.0,1.0,...,191837,1.0,0.021277,0.0,0.000000,0.0,0.00000,2015,10,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197920,20.238300,-155.837000,2,28.0,27.0,28.0,28.0,0.0,0.0,2.0,...,198121,2.0,1.000000,NaN,0.000000,0.0,0.00000,2018,1,22
197921,19.512500,-154.922000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,198121,1.0,1.000000,NaN,0.000000,0.0,3.49192,2018,2,9
197922,19.200000,-155.480000,2,33.0,33.0,33.0,33.0,0.0,0.0,1.0,...,198121,1.0,NaN,NaN,0.000000,0.0,0.00000,2018,1,7
197923,19.112700,-155.757000,2,32.0,31.0,42.0,64.0,0.0,0.0,3.0,...,198121,3.0,1.000000,NaN,1.000000,0.0,0.00000,2018,3,5


In [30]:
scaler = StandardScaler()
X = scaler.fit_transform(df_numeric.values)

array([[ 0.30655371,  0.888775  , -0.48683991, ...,  1.00560823,
        -1.24756739, -1.32840991],
       [ 0.30715146,  0.88869653, -0.6056945 , ...,  0.16037   ,
        -1.53716019, -0.19629843],
       [-0.26940972, -0.44428313, -0.84340367, ...,  1.00560823,
         1.6483606 ,  0.70939075],
       ...,
       [-3.55137873, -4.57296428, -0.72454908, ...,  1.85084647,
        -1.53716019, -0.98877647],
       [-3.56828662, -4.59217043, -0.72454908, ...,  1.85084647,
        -0.95797459, -1.21519876],
       [-3.11669303, -4.76058822, -0.72454908, ...,  1.85084647,
        -1.53716019,  1.38865763]])